In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

This notebook processes data for ADD/ADH calculation using scripts:
* create_weather_related_features.py
* create_weather_related_features_parallelized.py (faster)

In [7]:
# import data
#data = pd.read_pickle('../data/Gelderman_SOD_cohort/for_pmi_estimation.pkl')[['new_id','donor_date', 'correct_img_date',
       #'date_placed_ARF', 'PMI_days']]
    
#data = pd.read_csv('../data/Gelderman_SOD_cohort/unique_img_triplets_plus.csv')[['new_id','donor_date', 'correct_img_date',
       #'date_placed_ARF', 'PMI_days']]
data = pd.read_pickle('../data/master_dataset.pkl')[['new_id', 'correct_img_date', 'img',
       'date_placed_ARF', 'PMI_days']]
print(data.shape)
display(data.head())
display(data.info())

(1132970, 5)


,new_id,correct_img_date,img,date_placed_ARF,PMI_days
0,000,2018-01-21,00000121.01.JPG,2018-01-21,0.0
1,000,2018-01-21,00000121.02.JPG,2018-01-21,0.0
2,000,2018-01-21,00000121.03.JPG,2018-01-21,0.0
3,000,2018-01-21,00000121.04.JPG,2018-01-21,0.0
4,000,2018-01-21,00000121.05.JPG,2018-01-21,0.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132970 entries, 0 to 1132969
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   new_id            1132970 non-null  object 
 1   correct_img_date  1132970 non-null  object 
 2   img               1132970 non-null  object 
 3   date_placed_ARF   1132970 non-null  object 
 4   PMI_days          1132970 non-null  float64
dtypes: float64(1), object(4)
memory usage: 51.9+ MB


None

In [8]:
# extract donor_date from img column
data['donor_date'] = data.img.str.split('.', expand=True)[0]
display(data.sample(3))

,new_id,correct_img_date,img,date_placed_ARF,PMI_days,donor_date
242613,0fd,2016-06-12,0fd10612.40.JPG,2016-02-10,123.0,0fd10612
891199,c7e,2016-03-09,c7e00309.29.JPG,2016-02-10,28.0,c7e00309
234908,0f8,2020-03-23,0f800323.01.JPG,2020-02-14,38.0,0f800323


In [9]:
data['date_placed_ARF'] = pd.to_datetime(data['date_placed_ARF'], format='%Y-%m-%d')
data['correct_img_date'] = pd.to_datetime(data['correct_img_date'],format='%Y-%m-%d')
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132970 entries, 0 to 1132969
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   new_id            1132970 non-null  object        
 1   correct_img_date  1132970 non-null  datetime64[ns]
 2   img               1132970 non-null  object        
 3   date_placed_ARF   1132970 non-null  datetime64[ns]
 4   PMI_days          1132970 non-null  float64       
 5   donor_date        1132970 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 60.5+ MB


None

In [10]:
data.sort_values('donor_date').head(10)

,new_id,correct_img_date,img,date_placed_ARF,PMI_days,donor_date
0,000,2018-01-21,00000121.01.JPG,2018-01-21,0.0,00000121
1,000,2018-01-21,00000121.02.JPG,2018-01-21,0.0,00000121
2,000,2018-01-21,00000121.03.JPG,2018-01-21,0.0,00000121
3,000,2018-01-21,00000121.04.JPG,2018-01-21,0.0,00000121
4,000,2018-01-21,00000121.05.JPG,2018-01-21,0.0,00000121
5,000,2018-01-21,00000121.06.JPG,2018-01-21,0.0,00000121
6,000,2018-01-21,00000121.07.JPG,2018-01-21,0.0,00000121
7,000,2018-01-21,00000121.08.JPG,2018-01-21,0.0,00000121
8,000,2018-01-21,00000121.09.JPG,2018-01-21,0.0,00000121
24,000,2018-01-22,00000122.16.JPG,2018-01-21,1.0,00000122


In [11]:
unique_donor_date = data.drop_duplicates('donor_date').copy()
print(len(unique_donor_date))
display(unique_donor_date.head())
display(unique_donor_date.info())

38156


,new_id,correct_img_date,img,date_placed_ARF,PMI_days,donor_date
0,000,2018-01-21,00000121.01.JPG,2018-01-21,0.0,00000121
9,000,2018-01-22,00000122.01.JPG,2018-01-21,1.0,00000122
34,000,2018-01-23,00000123.01.JPG,2018-01-21,2.0,00000123
60,000,2018-01-24,00000124.01.JPG,2018-01-21,3.0,00000124
116,000,2018-01-25,00000125.01.JPG,2018-01-21,4.0,00000125


<class 'pandas.core.frame.DataFrame'>
Int64Index: 38156 entries, 0 to 1132967
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   new_id            38156 non-null  object        
 1   correct_img_date  38156 non-null  datetime64[ns]
 2   img               38156 non-null  object        
 3   date_placed_ARF   38156 non-null  datetime64[ns]
 4   PMI_days          38156 non-null  float64       
 5   donor_date        38156 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 2.0+ MB


None

In [12]:
# save unique_donor_date which will be used to calculate ADD
unique_donor_date.to_pickle('../data/Gelderman_SOD_cohort/unique_donor_date.pkl')

## After running create_weather_related_features_parallelized.py

In [32]:
with open('./../data/unique_donor_date_w_ADD.pkl', 'rb') as f:
        data_ls = pickle.load(f)
print(len(data_ls))

5


In [33]:
# convert list of dicts to dataframe
data_df = pd.DataFrame.from_dict(data_ls)
print(data_df.shape)
display(data_df.head())
display(data_df.info())
display(data_df.describe())

(5, 14)


,new_id,old_id,img,correct_img_date,date_placed_ARF,PMI_days,donor_date,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30
0,000,UT01-18D,00000121.01.JPG,2018-01-21,2018-01-21,0.0,00000121,8.645833,8.645833,0.000000,0,0,0,0
1,000,UT01-18D,00000122.01.JPG,2018-01-22,2018-01-21,1.0,00000122,19.072917,19.072917,10.427083,0,0,0,0
2,000,UT01-18D,00000123.01.JPG,2018-01-23,2018-01-21,2.0,00000123,28.018750,28.018750,10.427083,0,0,0,0
3,000,UT01-18D,00000124.01.JPG,2018-01-24,2018-01-21,3.0,00000124,31.940278,28.018750,10.427083,0,0,0,0
4,000,UT01-18D,00000125.01.JPG,2018-01-25,2018-01-21,4.0,00000125,35.281944,28.018750,10.427083,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   new_id            5 non-null      object        
 1   old_id            5 non-null      object        
 2   img               5 non-null      object        
 3   correct_img_date  5 non-null      datetime64[ns]
 4   date_placed_ARF   5 non-null      datetime64[ns]
 5   PMI_days          5 non-null      float64       
 6   donor_date        5 non-null      object        
 7   ADD_thres0        5 non-null      float64       
 8   ADD_thres5        5 non-null      float64       
 9   ADD_thres10       5 non-null      float64       
 10  ADD_thres15       5 non-null      int64         
 11  ADD_thres20       5 non-null      int64         
 12  ADD_thres25       5 non-null      int64         
 13  ADD_thres30       5 non-null      int64         
dtypes: datetime64[ns](2), float64(

None

,PMI_days,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30
count,5.000000,5.000000,5.000000,5.000000,5.0,5.0,5.0,5.0
mean,2.000000,24.591944,22.355000,8.341667,0.0,0.0,0.0,0.0
std,1.581139,10.779091,8.587018,4.663133,0.0,0.0,0.0,0.0
min,0.000000,8.645833,8.645833,0.000000,0.0,0.0,0.0,0.0
25%,1.000000,19.072917,19.072917,10.427083,0.0,0.0,0.0,0.0
50%,2.000000,28.018750,28.018750,10.427083,0.0,0.0,0.0,0.0
75%,3.000000,31.940278,28.018750,10.427083,0.0,0.0,0.0,0.0
max,4.000000,35.281944,28.018750,10.427083,0.0,0.0,0.0,0.0


In [48]:
data_df.sample(1)

,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,sex,age_at_death,est_stature_inches,est_weight,true_SOD,true_SOD_BP,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADH_thres0,ADH_thres5,ADH_thres10,ADH_thres15
608,082,UT06-13D,/da1_data/icputrd/arf/mean.js/public/anau_img3/082/08200224.24.JPG,08200224.24.JPG,2013-02-24,2013-01-23,32.0,1.0,59.0,182.88,220.0,3.0,torso,174.904965,153.600799,37.778611,0.0,4440.059167,3890.429167,1969.445,405.45


### Align with data

# Other
Testing purposes

In [ ]:
data_dict = data.to_dict('records')
row_counter = 0
# for each image
for i,row in enumerate(data_dict):
    print('iter:', i, 'ID:', row['new_id'])
    placement_date = row['date_placed_ARF']
    img_date = row['correct_img_date']

    ADD_thres0 = 0
    ADD_thres5 = 0
    ADD_thres10 = 0
    ADD_thres15 = 0
    
    ADH_thres0 = 0
    ADH_thres5 = 0
    ADH_thres10 = 0
    ADH_thres15 = 0
    while (placement_date <= img_date):
        print('   ', placement_date, img_date)
        # get weather for placement_date
        weather_current_df = weather_df[weather_df.date_time.dt.date == placement_date]
        #print(weather_current_df)

        # calculate ADD with varying temp thresholds
        avg_temp = weather_current_df.loc[:, 'HourlyDryBulbTemperature'].mean()
        print('     ', 'daily_avg_temp:', avg_temp)
        if avg_temp >= 0:
            ADD_thres0 += avg_temp
        if avg_temp >= 5:
            ADD_thres5 += avg_temp
        if avg_temp >= 10:
            ADD_thres10 += avg_temp
        if avg_temp >= 15:
            ADD_thres15 += avg_temp
        

        # calculate ADH with varying temp thresholds
        total_temp_thres0 = weather_current_df[weather_current_df.HourlyDryBulbTemperature >= 0]['HourlyDryBulbTemperature'].sum()
        ADH_thres0 += total_temp_thres0
        print('     ', 'daily_total_temp_thres0:', total_temp_thres0)
        
        total_temp_thres5 = weather_current_df[weather_current_df.HourlyDryBulbTemperature >= 5]['HourlyDryBulbTemperature'].sum()
        ADH_thres5 += total_temp_thres5
        print('     ', 'daily_total_temp_thres5:', total_temp_thres5)
        
        total_temp_thres10 = weather_current_df[weather_current_df.HourlyDryBulbTemperature >= 10]['HourlyDryBulbTemperature'].sum()
        ADH_thres10 += total_temp_thres10
        print('     ', 'daily_total_temp_thres10:', total_temp_thres10)
        
        total_temp_thres15 = weather_current_df[weather_current_df.HourlyDryBulbTemperature >= 15]['HourlyDryBulbTemperature'].sum()
        ADH_thres15 += total_temp_thres15
        print('     ', 'daily_total_temp_thres15:', total_temp_thres15)

        # go to the next day
        placement_date += datetime.timedelta(days=1)

    # add to dict
    row['ADD_thres0'] = ADD_thres0
    row['ADD_thres5'] = ADD_thres5
    row['ADD_thres10'] = ADD_thres10
    row['ADD_thres15'] = ADD_thres15
    
    row['ADH_thres0'] = ADH_thres0
    row['ADH_thres5'] = ADH_thres5
    row['ADH_thres10'] = ADH_thres10
    row['ADH_thres15'] = ADH_thres15
    print(row)

    print()
    print()

# convert dict to dataframe
#subset_w_weather_df = pd.DataFrame.from_dict(subset_dict)
# write to pickle
#subset_w_weather_df.to_pickle('./../data/subset_master_dataset_w_weather.pkl')